# Add Continuous Borders Around Labelled Regions

In [1]:
from freesurfer_surface import Surface

SUBJECTS_DIR = '../tests/subjects'
surface = Surface.read_triangular(SUBJECTS_DIR + '/fabian/surf/lh.pial')
surface.load_annotation_file(SUBJECTS_DIR + '/fabian/label/lh.aparc.annot')

In [2]:
import numpy
from freesurfer_surface import PolygonalChain, Vertex

def unit_vector(vector: numpy.array) -> numpy.array:
    return vector / numpy.linalg.norm(vector)

def add_blocks_along(border_chain: PolygonalChain, length_add_mm: float,
                     height_mm: float, width_mm: float):
    for vertex_triplet_indices in border_chain.adjacent_vertex_indices(3):
        vertex_triplet = surface.select_vertices(vertex_triplet_indices)
        backward_vector = vertex_triplet[0] - vertex_triplet[1]
        forward_vector = vertex_triplet[2] - vertex_triplet[1]
        forward_vector = unit_vector(forward_vector) \
                         * (numpy.linalg.norm(forward_vector) + length_add_mm)
        upward_vector = unit_vector(numpy.cross(backward_vector, forward_vector)) * height_mm / 2
        sideward_vector = unit_vector(numpy.cross(upward_vector, forward_vector)) * width_mm / 2
        base_corners = [vertex_triplet[1] + v for v in [-upward_vector -sideward_vector,
                                                        -upward_vector +sideward_vector,
                                                        +upward_vector +sideward_vector,
                                                        +upward_vector -sideward_vector]]
        top_corners = [v + forward_vector for v in base_corners]
        base_indices = [surface.add_vertex(Vertex(*coords)) for coords in base_corners]
        top_indices = [surface.add_vertex(Vertex(*coords)) for coords in top_corners]
        for surface_indices in [base_indices, top_indices]:
            surface.add_rectangle(surface_indices)
        for i in range(4):
            j = (i + 1) % 4
            surface.add_rectangle((base_indices[i], base_indices[j],
                                   top_indices[j], top_indices[i]))

In [3]:
for label in filter(lambda l: 'central' in l.name or 'parietal' in l.name,
                    surface.annotation.labels.values()):
    borders = list(surface.find_label_border_polygonal_chains(label))
    print('#{}: {} (borders: {})'.format(label.index, label.name, len(borders)))
    for border in borders:
        add_blocks_along(border, length_add_mm=0.2,
                         height_mm=1.5, width_mm=1.0)
            
surface.write_triangular('border.lh.pial')

#8: inferiorparietal (borders: 1)
#17: paracentral (borders: 1)
#22: postcentral (borders: 1)
#24: precentral (borders: 1)
#29: superiorparietal (borders: 1)


```sh
freeview --surface examples/border.lh.pial:annot=tests/subjects/fabian/label/lh.aparc.annot
```